# The Little Astronomy

The data in these tables come from 1) the Pinakes (<http://pinakes.irht.cnrs.fr/>) Greek manuscripts database and 2) a series of critical editions, enumerated in [one of the main tables](#The-Critical-Editions-and-Scholarly-Texts-discussing-the-11-texts-and-their-manuscript-witnesses) below.

### Contents
1. [First Steps](#First-Steps)
2. [The Imported CSV Files](#The-Imported-CSV-Files)
3. [The Main Tables](#The-Main-Tables)
4. [Querying the Dataset](#Querying-the-Dataset)

---------

## First Steps

In [46]:
import gzip            # can uncompress gzipped files, useful for accessing the pleiades data
import io              # useful routines for input/output
import numpy as np
import pandas as pd    # for working with "rows/columns" oriented data
import pymysql.cursors # 
import sqlite3         # an SQL database
import urllib.request  # for loading documents using http

In [47]:
# Connect to the database
connection = pymysql.connect(host='hosting.nyu.edu',
                             user='cmrougha_adsq',
                             password='####REPLACE####',
                             db='cmrougha_adsq2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

----------

## The Imported CSV Files

Nine CSV files were imported into Sequel Pro -- once there, the data in these files was restructured to produce the [Main Tables](#The-Main-Tables) outlined below. The originally imported tables do remain in the database. Their names are as follows:

* `x_citation_info`
* `x_dependencies`
* `x_ms_to_citation`
* `x_pinakes_all`
* `x_pinakes_corr`
* `x_recensions_crit-eds`
* `x_shelfmark_to_pinakes`
* `x_sigla_crit-eds`
* `x_work-and-author`

They can be queried in the below code block.

In [67]:
#Swap in any desired table name.
pd.read_sql("""
SELECT *
 FROM x_citation_info
""", connection)

,citation_old,citation_id,editor,year,citation_full
0,Moving Sphere (Hultsch 1885),hultsch1885,hultsch,1885,Autolycus of Pitane. Autolyci De Sphaera quae ...
1,Risings and Settings (Hultsch 1885),hultsch1885,hultsch,1885,Autolycus of Pitane. Autolyci De Sphaera quae ...
2,Autolycus (Hultsch 1885),hultsch1885,hultsch,1885,Autolycus of Pitane. Autolyci De Sphaera quae ...
3,Data (Menge 1896),menge1896,menge,1896,Euclid. Euclidis Data cum Commentario Marini e...
4,Optics (Heiberg 1895),heiberg1895,heiberg,1895,"Euclid. Euclidis Optica, Opticorum Recensio Th..."
5,Catoptrics (Heiberg 1895),heiberg1895,heiberg,1895,"Euclid. Euclidis Optica, Opticorum Recensio Th..."
6,Phaenomena (Menge 1916),menge1916,menge,1916,Euclid. Euclidis Phaenomena et Scripta Musica....
7,Anaphoricus (Hultsch 1888),manitius1888,hultsch,1888,[placeholder]
8,Sphaerica (Czinczenheim 2000),czinczenheim2000,czinczenheim,2000,"Theodosius. Edition, traduction, et commentair..."
9,Sphaerica (Heiberg 1927),heiberg1927,heiberg,1927,Theodosius. Theodosius Tripolites Sphaerica. E...


The main tables were produced from these with some manual correction, but largely via `CREATE TABLE` and SQL queries of the imported files. The following code block, for example, shows the query which was used to create one of the main tables, [`ms_contains-from-citations`](#Attestations-of-Manuscript-Witnesses-for-Texts-according-to-the-Critical-Editions).

In [ ]:
pd.read_sql("""
#CREATE TABLE `ms_contains-from-citations` AS ( 
 SELECT DISTINCT x_ms_to_citation.ms_id, x_ms_to_citation.text_id, sigla.sigla, x_ms_to_citation.citation_id
  FROM `x_ms_to_citation`
  LEFT JOIN ( 
   SELECT `x_sigla_crit-eds`.ms_id, `x_sigla_crit-eds`.sigla, `x_citation_info`.citation_id
    FROM `x_sigla_crit-eds`, `x_citation_info`
    WHERE ( `x_sigla_crit-eds`.citation_old = `x_citation_info`.citation_old )) AS sigla
  ON ( x_ms_to_citation.citation_id = sigla.citation_id )
  AND ( x_ms_to_citation.ms_id = sigla.ms_id )
# );
""", connection)

---------

## The Main Tables

### The 11 Extant Texts supposedly part of the Little Astronomy

Within these tables, the text is identified with one of the eleven `text_id`s. For readability, a shortened version of the English or Latin name is provided. This table also provides the appropriate Thesaurus Linguae Graecae identifier for each text and the title as it appears in the Pinakes database. In two cases the TLG has different identifiers for different recensions, and so that information is included as well.

In [48]:
pd.read_sql("""
SELECT *
 FROM texts
""", connection)

,text_id,text_name-short,text_tlg,recension_tlg,text_pinakes
0,text_S,Sphaerica,tlg1719.tlg001,,Sphaerica
1,text_H,Habitations,tlg1719.tlg002,,De habitationibus
2,text_NaD,Nights and Days,tlg1719.tlg003,,De diebus et noctibus
3,text_MS,Moving Sphere,tlg1210.tlg001,,De sphaera quae mouetur
4,text_RaS,Risings and Settings,tlg1210.tlg002,,De ortibus et occasibus
5,text_P,Phaenomena,tlg1799.tlg012,,Phaenomena
6,text_P,Phaenomena,tlg1799.tlg013,b,Phaenomena
7,text_D,Data,tlg1799.tlg007,,Data
8,text_O,Optica,tlg1799.tlg009,,Optica
9,text_O,Optica,tlg1799.tlg010,theonis,Optica


### The Authors of the Little Astronomy Texts

A similar pattern is followed here as in the table of texts. Data is incorporated from the TLG and from Pinakes.

In [49]:
pd.read_sql("""
SELECT *
 FROM authors
""", connection)

,author_id,author_name,author_tlg,author_pinakes
0,author_Th,Theodosius,tlg1719,Theodosius Tripolita
1,author_Au,Autolycus,tlg1210,Autolycus astronomus
2,author_Eu,Euclid,tlg1799,Euclides
3,author_Ar,Aristarchus,tlg1181,Aristarchus Samius astronomus
4,author_Hy,Hypsicles,tlg0717,Hypsicles


### Texts and Authors

In [50]:
pd.read_sql("""
SELECT *
 FROM `text-to-author`
""", connection)

,text_id,author_id
0,text_S,author_Th
1,text_H,author_Th
2,text_NaD,author_Th
3,text_MS,author_Au
4,text_RaS,author_Au
5,text_P,author_Eu
6,text_D,author_Eu
7,text_O,author_Eu
8,text_C,author_Eu
9,text_SaD,author_Ar


### The Critical Editions and Scholarly Texts discussing the 11 texts and their manuscript witnesses

Each is referenced via a `citation_id`; this table links this identifier to the full citation.

In [51]:
pd.read_sql("""
SELECT *
 FROM citations
""", connection)

,citation_id,editor,year,citation_full
0,hultsch1885,Hultsch,1885,Autolycus of Pitane. Autolyci De Sphaera quae ...
1,menge1896,Menge,1896,Euclid. Euclidis Data cum Commentario Marini e...
2,heiberg1895,Heiberg,1895,"Euclid. Euclidis Optica, Opticorum Recensio Th..."
3,menge1916,Menge,1916,Euclid. Euclidis Phaenomena et Scripta Musica....
4,manitius1888,Manitius,1888,Hypsicles. Des Hypsikles Schrift Anaphorikos n...
5,czinczenheim2000,Czinczenheim,2000,"Theodosius. Edition, traduction, et commentair..."
6,heiberg1927,Heiberg,1927,Theodosius. Theodosius Tripolites Sphaerica. E...
7,fecht1927,Fecht,1927,Theodosius. Theodosii De Habitationibus liber ...
8,noack1992,Noack,1992,Aristarch von Samos Untersuchungen zur Überlie...
9,mogenet1950,Mogenet,1950,"Autolycus of Pitane. Autolycus de Pitane, Hist..."


### Attestations of Manuscript Witnesses for Texts according to the Critical Editions

`ms_id` contains `text_id` according to `citation_id`. If a particular edition uses sigla to refer to particular manuscripts, that data is incorporated as well.

The full table contains 368 rows; the below query offers a random selection of 10 rows.

In [52]:
pd.read_sql("""
SELECT *
 FROM `ms_contains-from-citations`
 ORDER BY RAND()
 LIMIT 10
""", connection)

,ms_id,text_id,sigla,citation_id
0,Marc.gr.304,text_MS,M,mogenet1950
1,Vat.gr.190,text_D,None,menge1896
2,Par.gr.2342,text_MS,None,mogenet1950
3,Par.gr.2365,text_S,n,czinczenheim2000
4,Par.gr.2350,text_O,None,heiberg1895
5,Laur.Plut.28.14,text_RaS,None,mogenet1950
6,Bodl.Savile.10,text_SaD,W,noack1992
7,Par.gr.2364,text_MS,None,hultsch1885
8,Par.gr.2350,text_P,None,menge1916
9,Ambr.C263inf.,text_SaD,None,noack1992


### Attestations of MS Witnesses for particular Recensions according to the Critical Editions

The full table contains 93 rows; the below query offers a random selection of 10 rows.

In [53]:
pd.read_sql("""
SELECT *
 FROM `recensions-from-citations`
 ORDER BY RAND()
 LIMIT 10
""", connection)

,ms_id,text_id,recension_crit-ed,citation_id
0,Vat.Barb.gr.260,text_D,Theon (part),menge1896
1,Par.gr.2342,text_O,Theon,heiberg1895
2,Laur.Plut.28.6,text_P,recension a,menge1916
3,Ambr.J84inf.,text_P,recension b,menge1916
4,S.John.55,text_P,recension b,menge1916
5,Vat.gr.1039,text_O,genuinis,heiberg1895
6,Marc.gr.302,text_P,recension b,menge1916
7,Par.gr.2366,text_O,Theon,heiberg1895
8,Laur.Plut.28.3,text_P,recension a,menge1916
9,Ambr.A101sup.,text_O,Theon,heiberg1895


### Manuscript Libraries

Information drawn from Pinakes.

In [54]:
pd.read_sql("""
SELECT *
 FROM libraries
""", connection)

,loc_institution,loc_city,loc_country
0,Biblioteca Riccardiana,Firenze,Italia
1,Bibliothèque nationale de France (BNF),Paris,France
2,Biblioteca comunale degli Intronati,Siena,Italia
3,Bibliotheek der Rijksuniversiteit,Leiden,Nederland
4,Biblioteca Nazionale Centrale,Firenze,Italia
5,Biblioteca Apostolica Vaticana,Vaticano,Vaticano
6,Biblioteca Nazionale Marciana,Venezia,Italia
7,Trinity College,Cambridge,United Kingdom
8,Biblioteca Ambrosiana,Milano,Italia
9,Real Biblioteca,Escorial (El-),España


### Location of Manuscripts

Information drawn from Pinakes. Some gaps resulted in the transition which are to be corrected.

The full table contains 155 rows; again a random selection is provided.

In [55]:
pd.read_sql("""
SELECT *
 FROM `ms-to-library`
 ORDER BY RAND()
 LIMIT 10
""", connection)

,shelfmark_id,loc_institution,pinakes_fonds,pinakes_cotes
0,Scorial.Gamma-II-13,None,None,None
1,Par.suppl.gr.13,None,None,None
2,Vat.gr.203,None,None,None
3,Par.gr.2347,Bibliothèque nationale de France (BNF),gr.,2347
4,Par.gr.2506,Bibliothèque nationale de France (BNF),gr.,2506
5,Trinity.O.5.15,Trinity College,fonds principal,O.05.15 (1296)
6,Ambr.C263inf.,Biblioteca Ambrosiana,fonds principal,C 263 inf. (Martini-Bassi 0903)
7,Ambr.Q105sup.,Biblioteca Ambrosiana,fonds principal,Q 105 sup. (Martini-Bassi 697)
8,Wolfen.Gud.gr.37,Herzog August Bibliothek,Gud. gr.,Gud. gr. 037
9,Balliol.267,Balliol College,fonds principal,267


### Manuscript Witnesses for Texts according to Pinakes

The below table is simplified from the csv files exported from Pinakes to include only the information that is relevant for the relation between each manuscript and the text from the Little Astronomy collection which it contains. `pinakes_century` is included in this table because the Pinakes dataset does show some occasional variety in date within the same manuscript -- the dates are dates for the texts, not the manuscripts as a whole.

Very few recensions and comments are included, but they are present and are dependent on the text, not the manuscript.

`shelfmark_name-short` is temporarily included in this table because some `shelfmark_id`s were lost in the transition and restructuring of these tables -- they are to be re-added with reference to the old csv files.

The full table has 436 rows.

In [56]:
pd.read_sql("""
SELECT *
 FROM `ms_contains-from-pinakes`
 ORDER BY RAND()
 LIMIT 10
""", connection)

,shelfmark_id,shelfmark_name-short,pinakes_century,text_id,pinakes_folios,pinakes_recensions,pinakes_comments
0,None,Real Biblioteca fonds principal ?. I. 04 (Andr...,16,text_MS,113-121v,,
1,Leiden.BPG.7,Bibliotheek der Rijksuniversiteit BPG 7,16,text_C,320-332v,,
2,Vat.Pal.gr.62,Biblioteca Apostolica Vaticana Pal. gr. 62,16,text_S,001-36*,,
3,Barnard.8,Coll. Ch. Humberd fonds principal 8,17,text_MS,123-124*,,
4,Phillipps.1543,"Thirlestaine House, coll. Phillipps fonds prin...",,text_C,,,
5,Par.gr.2506,Bibliothèque nationale de France (BNF) gr. 2506,14,text_NaD,057r-v*,,
6,None,Universitätsbibliothek fonds principal M. ch. ...,15 ex. - 16 in.,text_C,045-47,,
7,Marc.gr.XI.30f,Biblioteca Nazionale Marciana gr. XI. 030f (co...,16 ex.,text_RaS,115-133,,"Ed. F. Hultsch, Lipsiae 1885, pp. 48-159"
8,Laur.Plut.28.6,Biblioteca Medicea Laurenziana Plut. 28. 06,13,text_O,,,
9,Vat.gr.1316,Biblioteca Apostolica Vaticana Vat. gr. 1316,,text_O,,,


### Manuscript Shelfmarks

The `shelfmark_id`s were created during the collection of this data. The remaining information comes from Pinakes.

Based on the display of the `pinakes_id`s below, it might be more appropriate to choose a different datatype for that column.

The full table has 155 rows.

In [57]:
pd.read_sql("""
SELECT *
 FROM mss
 ORDER BY RAND()
 LIMIT 10
""", connection)

,shelfmark_id,pinakes_id,shelfmark_name-short,shelfmark_name-long
0,Bayer.Staatsbibl.gr.361,44809.0,Bayerische Staatsbibliothek Cod.graec. 361,Deutschland München Bayerische Staatsbibliothe...
1,Par.gr.2365,51997.0,Bibliothèque nationale de France (BNF) gr. 2365,France Paris Bibliothèque nationale de France ...
2,Ambr.P270sup.,43132.0,Biblioteca Ambrosiana fonds principal P 270 su...,Italia Milano Biblioteca Ambrosiana fonds prin...
3,Par.gr.2473,52105.0,Bibliothèque nationale de France (BNF) gr. 2473,France Paris Bibliothèque nationale de France ...
4,Toletani.29,NaN,None,None
5,ÖNB.phil.gr.268,71382.0,Österreichische Nationalbibliothek (ÖNB) phil....,Österreich Wien Österreichische Nationalbiblio...
6,Laur.Plut.28.8,16189.0,Biblioteca Medicea Laurenziana Plut. 28. 08,Italia Firenze Biblioteca Medicea Laurenziana ...
7,Vat.Ottob.gr.102,65343.0,Biblioteca Apostolica Vaticana Ottob. gr. 102,Vaticano Vaticano Biblioteca Apostolica Vatica...
8,Ambr.A194inf.,42277.0,Biblioteca Ambrosiana fonds principal A 194 in...,Italia Milano Biblioteca Ambrosiana fonds prin...
9,Phillipps.1542,12571.0,Thirlestaine House coll. Phillipps fonds princ...,United Kingdom Cheltenham Thirlestaine House c...


### Manuscripts, Number of Texts Contained Overall, and Number of Little Astronomy Texts Contained

Information drawn from Pinakes and corrected. Information from the critical editions not present in the Pinakes dataset has also been added.

The full table contains 147 rows.

In [58]:
pd.read_sql("""
SELECT *
 FROM `ms-to-texts`
 ORDER BY RAND()
 LIMIT 10
""", connection)

,shelfmark_id,number_all,number_LA,text_S,text_MS,text_O,text_P,text_H,text_NaD,text_SaD,text_RaS,text_A,text_C,text_D,corrected
0,Laur.Plut.28.1,8,1,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,
1,Siena.L.X.56,15,7,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,!
2,Par.gr.2364,5,5,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,FALSE,
3,Vat.gr.1316,2,1,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,
4,Phillipps.1543,1,1,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,
5,Hagion.727,1,1,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,
6,Par.gr.2352,5,4,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,
7,Laur.Plut.28.14,16,1,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,!
8,Vat.gr.193,7,1,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,!
9,Vat.Ottob.gr.102,3,2,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,


### Manuscript Dependencies According to the Critical Editions

The prolegomena of the critical editions very often note relations between the manuscripts and sometimes include stemmata -- the following table has extracted that information.

The full table contains 190 rows.

In [59]:
pd.read_sql("""
SELECT *
 FROM `dependencies-from-citations`
 ORDER BY RAND()
 LIMIT 10
""", connection)

,MS A,comment,[A] [verb] ex [D],MS D,videtur,citation_id
0,Par.gr.2363,,descendant,Par.gr.2472,,czinczenheim2000
1,Cambridge.Gg.II.33,,pendet,Vat.gr.202,,heiberg1895
2,Phillipps.1543,ex ipso,descriptus est,Marc.gr.301,,heiberg1895
3,Angel.gr.95,,descriptus est,Vat.gr.192,,heiberg1895
4,Par.gr.2390,,originem ducit,Vat.gr.204,,heiberg1895
5,Par.gr.2365,,[line from],Vat.gr.202,,mogenet1950
6,Vat.gr.191,priusquam interpolaretur,fluit,Vat.gr.204,,menge1896
7,Ambr.Q112sup.,,descendant,Vat.gr.191,,czinczenheim2000
8,Vat.Barb.gr.260,fortasse,descriptus est,Vat.gr.192,,heiberg1895
9,Par.gr.2107,,descriptus est,Vat.gr.204,,heiberg1895


-----------

## Querying the Dataset

The following query pulls out the 21 manuscripts which contain all four of the Euclidean texts and looks to see to what extent they include also the other supposed Little Astronomy texts. Data has been drawn from Pinakes regarding the total number of texts in each manuscript and how many of that total are Little Astronomy texts. Here our resulting output is ordered according to which of this subset of manuscripts contain the highest percentage of Little Astronomy texts.

In [60]:
pd.read_sql("""
SELECT shelfmark_id, number_LA, number_all, (number_LA / number_all)*100 AS '% LA Texts / All Texts', text_S AS 'Sphaerica', text_H AS 'Habitations', text_NaD AS 'Nights and Days', text_MS AS 'Moving Sphere', text_RaS AS 'Risings and Settings', text_SaD AS 'Sizes and Distances', text_A AS 'Anaphoricus'
 FROM `ms-to-texts`
 WHERE text_O = 'TRUE'
 AND text_P = 'TRUE'
 AND text_C = 'TRUE'
 AND text_D = 'TRUE'
 ORDER BY `% LA Texts / All Texts` DESC;
""", connection)

,shelfmark_id,number_LA,number_all,% LA Texts / All Texts,Sphaerica,Habitations,Nights and Days,Moving Sphere,Risings and Settings,Sizes and Distances,Anaphoricus
0,Toledo.98-13,4,4,100.0000,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
1,Phillipps.1542,4,4,100.0000,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
2,Vat.Ross.978,11,12,91.6667,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
3,Par.gr.2366,9,10,90.0000,TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE
4,Vat.gr.204,11,13,84.6154,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
5,Par.gr.2352,4,5,80.0000,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
6,Beinecke.424,4,5,80.0000,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
7,Scorial.Chi-I-4,10,14,71.4286,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
8,Par.gr.2347,5,7,71.4286,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE
9,Par.gr.2342,11,19,57.8947,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE


The following two queries take a look at the sigla used by the editors of the critical editions. Two hypotheses are supposed and the data sought below: 1) the siglum A will often be given to the oldest manuscript and 2) the siglum A will be given to a manuscript from a library whose name starts with that letter.

In [62]:
pd.read_sql("""
SELECT `ms_contains-from-citations`.ms_id, `ms_contains-from-pinakes`.pinakes_century, `ms_contains-from-citations`.sigla, `ms_contains-from-citations`.text_id, `ms_contains-from-citations`.citation_id
 FROM `ms_contains-from-citations`, `ms_contains-from-pinakes`
 WHERE `ms_contains-from-citations`.sigla = 'A'
 AND `ms_contains-from-citations`.text_id = `ms_contains-from-pinakes`.text_id
 AND `ms_contains-from-citations`.ms_id = `ms_contains-from-pinakes`.shelfmark_id
 AND `ms_contains-from-pinakes`.pinakes_century != ''
 ORDER BY `ms_contains-from-pinakes`.pinakes_century ASC;
""", connection)

,ms_id,pinakes_century,sigla,text_id,citation_id
0,Vat.gr.204,10,A,text_S,czinczenheim2000
1,Vat.gr.204,10,A,text_S,heiberg1927
2,Vat.gr.204,10,A,text_SaD,noack1992
3,Vat.gr.191,13,A,text_MS,hultsch1885
4,Laur.Plut.28.1,13,a,text_D,menge1896
5,Vat.gr.191,13,A,text_RaS,hultsch1885
6,Ambr.A92sup.,15-16,A,text_O,heiberg1895
7,Ambr.A101sup.,16,a,text_S,czinczenheim2000
8,Leiden.Scal.gr.39,16,a,text_A,defalco1966
9,Ambr.A101sup.,16,A,text_A,manitius1888


In [63]:
pd.read_sql("""
SELECT `ms_contains-from-citations`.ms_id, `ms_contains-from-pinakes`.pinakes_century, `ms_contains-from-citations`.sigla, `ms_contains-from-citations`.text_id, `ms_contains-from-citations`.citation_id
 FROM `ms_contains-from-citations`, `ms_contains-from-pinakes`, mss
 WHERE `ms_contains-from-citations`.sigla = 'A'
 AND `ms_contains-from-citations`.text_id = `ms_contains-from-pinakes`.text_id 
 AND `ms_contains-from-citations`.ms_id = `ms_contains-from-pinakes`.shelfmark_id
 AND `ms_contains-from-pinakes`.pinakes_century != ''
 AND ( mss.shelfmark_id = `ms_contains-from-citations`.ms_id )
 AND ( mss.shelfmark_id LIKE 'A%'
 OR mss.`shelfmark_name-short` LIKE 'A%' );
""", connection)

,ms_id,pinakes_century,sigla,text_id,citation_id
0,Ambr.A101sup.,16,A,text_A,manitius1888
1,Ambr.A101sup.,16,A,text_A,defalco1966
2,Ambr.A101sup.,16,A,text_RaS,mogenet1950
3,Ambr.A101sup.,16,A,text_MS,mogenet1950
4,Ambr.A92sup.,15-16,A,text_O,heiberg1895
5,Ambr.A101sup.,16,a,text_S,czinczenheim2000


The below query is supposed to look at the manuscript dependencies table and pull in the dates (according to Pinakes) for each manuscript via their `shelfmark_id` so that dates can be compared to check for conflicts (such as a descendant MS being of earlier date than an ancestor MS).

The query uses one of the direct-from-csv tables, `x_pinakes_corr`, because it was found that some useful data was overlooked in the tables' restructuring in this case.

Strangely, the query does not appear to work in the Jupyter notebook. The obtained century information for each manuscript is the same here, and is the date for only the `Possible Descendant` manuscript. In Sequel Pro the query runs correctly. The reason for this issue has not yet been determined.

In [65]:
pd.read_sql("""
SELECT DISTINCT x.`Possible Descendant`, x.century AS 'Pinakes Century', x.`Possible Ancestor`, `x_pinakes_corr`.century AS 'Pinakes Century', x.citation_id
 FROM ( 
  SELECT DISTINCT `dependencies-from-citations`.`MS A` AS 'Possible Descendant', `dependencies-from-citations`.`MS D` AS 'Possible Ancestor', `x_pinakes_corr`.century, `dependencies-from-citations`.citation_id
   FROM `dependencies-from-citations`, `ms_contains-from-pinakes`, `x_pinakes_corr`, mss
   WHERE ( `dependencies-from-citations`.`[A] [verb] ex [D]` NOT LIKE '%non%' )
   AND ( `dependencies-from-citations`.`MS A` = mss.`shelfmark_id`
   AND mss.`shelfmark_name-long` = `x_pinakes_corr`.shelfmark_long )
 ) AS x, `x_pinakes_corr`, mss
 WHERE ( x.`Possible Ancestor` = mss.`shelfmark_id`
 AND mss.`shelfmark_name-long` = `x_pinakes_corr`.shelfmark_long )
 ORDER BY RAND()
 LIMIT 10
 """, connection)

,Possible Descendant,Pinakes Century,Possible Ancestor,Pinakes Century,citation_id
0,Ambr.A101sup.,16/15-16,Par.gr.2342,16/15-16,heiberg1927
1,Scorial.Chi-I-4,16,Marc.gr.304,16,defalco1966
2,ÖNB.suppl.gr.9,16/16 med.,Vat.gr.204,16/16 med.,defalco1966
3,Par.suppl.gr.186,16,Vat.gr.191,16,heiberg1895
4,Scorial.Y-I-7,16,Vat.gr.204,16,defalco1966
5,Par.gr.2347,16,Vat.gr.192,16,heiberg1895
6,S.John.55,16,Bayer.Staatsbibl.gr.361,16,menge1916
7,Ambr.Q112sup.,16,Vat.gr.191,16,czinczenheim2000
8,Par.gr.2013,16,Marc.gr.301,16,heiberg1895
9,Par.gr.2386,16,Par.gr.2364,16,noack1992
